In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive/CS3237_ML_Training")

In [3]:
from google.colab import output
output.enable_custom_widget_manager()

In [60]:
"""
Preprocessing of data, I'll use the 11/5 csv for now
cut data off at 18.00.58 (row 1 : 101 inclusive) (pics img 4 : 104 inclusive) - 5a one
(row 901 : 1047 inclusive) (pics img 1 : 147 inclusive) - 5b one
these should be treated as separate sequences and processed as such then combined
Data is Time : windSpeed : bmpTemp : bmpPressure : dhtTemp : dhtHumidity : isRaining
"""
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

df = pd.read_csv('CollectedData/11_5_24.csv', header=None)
data_11_5a = df.iloc[0:101, 1:] # should be equivalent in data to the images
data_11_5b = df.iloc[900:1047, 1:] # should be equivalent in data to the images
print(data_11_5a)
print(data_11_5b)

        1     2       3     4   5  6
0    0.23  31.1  100243  29.1  69  0
1    0.30  31.3  100244  29.1  69  0
2    0.31  31.5  100245  29.2  69  0
3    0.34  31.7  100242  29.3  69  0
4    0.54  31.8  100239  29.4  69  0
..    ...   ...     ...   ...  .. ..
96   0.40  31.8  100237  30.4  60  0
97   0.80  31.7  100244  30.4  60  0
98   0.99  31.7  100250  30.4  60  0
99   0.61  31.6  100249  30.3  60  0
100  1.00  31.6  100248  30.3  60  0

[101 rows x 6 columns]
         1     2       3     4   5  6
900   0.55  25.3  100750  24.0  87  1
901   1.10  25.2  100764  24.0  87  1
902   0.72  25.1  100753  24.0  87  1
903   0.64  25.0  100765  24.0  87  1
904   0.76  24.9  100775  24.0  87  1
...    ...   ...     ...   ...  .. ..
1042  0.47  26.8  100808  25.4  91  0
1043  0.23  26.9  100804  25.4  91  1
1044  0.59  27.0  100804  25.4  91  1
1045  0.47  27.1  100803  25.5  90  1
1046  0.18  27.1  100805  25.5  90  1

[147 rows x 6 columns]


In [61]:
transform = transforms.Compose([
  transforms.Resize((128, 128)),
  transforms.ToTensor(),
  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
imgData1 = datasets.ImageFolder(root='CollectedData/11_5a_24', transform=transform)
imgLoader1 = DataLoader(imgData1, shuffle=False)
imgData2 = datasets.ImageFolder(root='CollectedData/11_5b_24', transform=transform)
imgLoader2 = DataLoader(imgData2, shuffle=False)

In [11]:
print(imgLoader1.dataset[0][0].shape)

torch.Size([3, 128, 128])


In [62]:
imgs_5a = torch.zeros((101, 3, 128, 128))
for i in range (101):
  imgs_5a[i] = imgLoader1.dataset[i][0]

imgs_5b = torch.zeros((147, 3, 128, 128))
for i in range (147):
  imgs_5b[i] = imgLoader2.dataset[i][0]
print(imgs_5a)
print(imgs_5b)

tensor([[[[-0.2235, -0.2314, -0.2392,  ..., -0.6314, -0.6157, -0.5922],
          [-0.0196, -0.0275, -0.0353,  ..., -0.5529, -0.5451, -0.5373],
          [ 0.1059,  0.0980,  0.0902,  ..., -0.4980, -0.5059, -0.5137],
          ...,
          [ 0.2863,  0.2941,  0.3098,  ..., -0.3255, -0.4275, -0.5529],
          [ 0.2863,  0.2941,  0.3098,  ..., -0.1529, -0.3020, -0.5451],
          [ 0.2863,  0.2941,  0.3098,  ..., -0.1529, -0.2863, -0.5451]],

         [[-0.0431, -0.0510, -0.0588,  ..., -0.5843, -0.5843, -0.5608],
          [ 0.0588,  0.0510,  0.0431,  ..., -0.5529, -0.5608, -0.5451],
          [ 0.0824,  0.0745,  0.0667,  ..., -0.5373, -0.5451, -0.5451],
          ...,
          [ 0.2863,  0.2941,  0.3098,  ..., -0.3725, -0.4745, -0.6000],
          [ 0.2863,  0.2941,  0.3098,  ..., -0.2000, -0.3490, -0.5922],
          [ 0.2863,  0.2941,  0.3098,  ..., -0.2000, -0.3333, -0.5922]],

         [[ 0.1686,  0.1608,  0.1529,  ..., -0.4039, -0.4196, -0.4118],
          [ 0.2000,  0.1922,  

In [17]:
# Define the CNN model
class WeatherModel(nn.Module):
  def __init__(self, num_classes):
    super(WeatherModel, self).__init__()
    self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
    self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
    self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)

    self.fc1 = nn.Linear(128 * 8 * 8, 256)
    self.dropout = nn.Dropout(0.5)
    self.fc2 = nn.Linear(256, num_classes)

  def forward(self, x):
    x = self.pool(torch.relu(self.conv1(x)))
    x = self.pool(torch.relu(self.conv2(x)))
    x = self.pool(torch.relu(self.conv3(x)))
    x = self.pool(torch.relu(self.conv4(x)))

    x = torch.flatten(x, 1)
    x = torch.relu(self.fc1(x))
    x = self.dropout(x)
    x = self.fc2(x)

    return x

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = WeatherModel(num_classes=6).to(device)
model.load_state_dict(torch.load("WeatherModel.pth", map_location=torch.device('cpu')))
model.eval()

<ipython-input-20-3cfefd94fea2>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("WeatherModel.pth", map_location=torch.device('cpu')))


WeatherModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=8192, out_features=256, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=256, out_features=6, bias=True)
)

In [41]:
img_result_5a = model(imgs_5a.to(device))
img_result_5b = model(imgs_5b.to(device))
_, num_result_5a = torch.max(img_result_5a, 1)
_, num_result_5b = torch.max(img_result_5b, 1)
print(num_result_5a)
print(num_result_5b)

tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5,
        5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 5, 4, 4])
tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5])


In [63]:
print(num_result_5a.shape)
target_11_5a = torch.tensor(data_11_5a.iloc[:, 5].values)
dataset_11_5a = torch.cat((num_result_5a.reshape((101, 1)), torch.tensor(data_11_5a.iloc[:, 0:5].values)), dim = 1)
print(dataset_11_5a)
target_11_5b = torch.tensor(data_11_5b.iloc[:, 5].values)
dataset_11_5b = torch.cat((num_result_5b.reshape((147, 1)), torch.tensor(data_11_5b.iloc[:, 0:5].values)), dim = 1)
print(dataset_11_5b)

torch.Size([101])
tensor([[5.0000e+00, 2.3000e-01, 3.1100e+01, 1.0024e+05, 2.9100e+01, 6.9000e+01],
        [5.0000e+00, 3.0000e-01, 3.1300e+01, 1.0024e+05, 2.9100e+01, 6.9000e+01],
        [5.0000e+00, 3.1000e-01, 3.1500e+01, 1.0024e+05, 2.9200e+01, 6.9000e+01],
        [5.0000e+00, 3.4000e-01, 3.1700e+01, 1.0024e+05, 2.9300e+01, 6.9000e+01],
        [5.0000e+00, 5.4000e-01, 3.1800e+01, 1.0024e+05, 2.9400e+01, 6.9000e+01],
        [5.0000e+00, 1.0400e+00, 3.1700e+01, 1.0024e+05, 2.9600e+01, 6.8000e+01],
        [5.0000e+00, 1.4000e-01, 3.1700e+01, 1.0024e+05, 2.9800e+01, 6.7000e+01],
        [5.0000e+00, 2.8000e-01, 3.1800e+01, 1.0025e+05, 2.9800e+01, 6.6000e+01],
        [5.0000e+00, 2.5000e-01, 3.1800e+01, 1.0024e+05, 2.9800e+01, 6.5000e+01],
        [5.0000e+00, 3.5000e-01, 3.2000e+01, 1.0024e+05, 2.9900e+01, 6.5000e+01],
        [5.0000e+00, 2.7000e-01, 3.2200e+01, 1.0024e+05, 2.9900e+01, 6.5000e+01],
        [5.0000e+00, 4.9000e-01, 3.2300e+01, 1.0023e+05, 2.9900e+01, 6.5000e+01]

In [64]:
# saving processed data for the next step
torch.save(dataset_11_5a, 'dataset_11_5a.pt')
torch.save(dataset_11_5b, 'dataset_11_5b.pt')
torch.save(target_11_5a, 'target_11_5a.pt')
torch.save(target_11_5b, 'target_11_5b.pt')